<a href="https://colab.research.google.com/github/MadBunny999/Time-Serios-Exam/blob/main/ExamReport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tbats sktime==0.31.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.0/24.0 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.2/136.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 57.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import joblib

from sklearn.metrics import r2_score

from sktime.performance_metrics.forecasting import MeanSquaredError, MeanAbsoluteError
from sktime.forecasting.arima import ARIMA
from sktime.forecasting.ets import AutoETS

import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


# ЗАДАНИЕ: Прогнозирование продаж в магазинах офлаин-ритейлера в США <a class="tocSkip">

В качестве экзаменационного задания вам предстоит построить модель прогнозирования спроса(продаж) на товары в магазинах офлаин-ритейлера в США . Всего в датасете 3 магазина, в каждом магазине по 15 артикулов (товаров). Вам нужно выбрать 1 магазин (любой из 3).

Прогнозировать продажи нужно на неделю, на месяц и на квартал.

В качестве дополнительной информации вам переданы данные о цене товара (меняются раз в неделю, а также о праздниках в США).

Задание 1. Реализовать класс, который умеет:

1) предобрабатывать исходные данные в удобный формат;

2) обучаться для задачи прогнозирования;

3) оценивать качество своих прогнозов;

4) сохранять модели и подгружать их;

5) Прогнозировать продажи на неделю, на месяц и на квартал.

Должна быть рабочая программа, которая делает инференс (прогнозирование на произвольном тестовом датасете (аналогично тому, который есть у вас)).


Задача 2. Подготовить отчёт о решении данной задачи в виде jupyter ноутбука. В отчёте, в частности, вы должны ответить на следующие вопросы:

1) Какие методы предобработки данных вы использовали?

2) Какие модели пробовали? Почему пробовали именно их?

3) Как вы проверяете качество модели? На каких данных? Какие метрики используюте? Чем обусловлен выбор именно этих метрик?

4) Какое итоговое качество модели на тестовом датасете?

# Задание 1: Реализация класса

In [ ]:
class TimeSeriosProcessor:

    def __init__(self, sales, prices, calendar):

        self.data = pd.merge(pd.merge(sales,calendar,how='left',
                                      on = 'date_id').drop(columns=['weekday',
                                                                    'CASHBACK_STORE_2']),prices,how='left',
                                                                     on = ['item_id', 'wm_yr_wk'])
        self.data["date"] = pd.to_datetime(self.data["date"])
        self.shorttime_models = [ARIMA(order=(7, 0, 5), suppress_warnings=True) for i in enumerate(self.data.item_id.unique())]
        self.longtime_models = [AutoETS(auto=True, sp=7, n_jobs=-1) for i in enumerate(self.data.item_id.unique())]

    def fit(self):

        for i, art in enumerate(self.data.item_id.unique()):
            train_data = self.data[self.data.item_id == art].set_index('date')['cnt']
            self.shorttime_models[i].fit(train_data)
            self.longtime_models[i].fit(train_data)
        return self

    def predict(self, period = 'Week'):

        periods = {'Week':7,
                   'Month':30,
                   'Quarter':90}
        predicts = {}
        key = 0

        for i, art in enumerate(self.data.item_id.unique()):
            data_test = self.data[self.data['item_id']==art][['date','cnt']].set_index('date')
            last_date = pd.to_datetime(str(data_test.iloc[-1:].index[0])[:-9])
            dates = pd.date_range(last_date+pd.Timedelta('1 Days'),last_date+pd.Timedelta(f'{periods[period]} Days'))
            key = art

            if period == 'Week':
                predicts[art] = self.longtime_models[i].predict(dates)
            else:
                predicts[art] = self.shorttime_models[i].predict(dates)
        return pd.concat(predicts, axis=1).reindex(predicts[key].index)

    def metrics_report(self, y_true,y_pred):

        mae_func = MeanAbsoluteError()
        mse_func = MeanSquaredError()

        maes = mae_func.evaluate_by_index(y_true,y_pred)
        mses = mse_func.evaluate_by_index(y_true,y_pred)

        stats_list = ['mean','median']

        res = pd.DataFrame([maes.agg(stats_list),
                            mses.agg(stats_list)],
                            index=['maes','mses'])
        return res


    def eval(self):

        res_week, res_month, res_quarter = [],[],[]

        for i, art in enumerate(self.data.item_id.unique()):
            ans = self.data[self.data['item_id']==art]['cnt']
            dates = self.data[self.data['item_id']==art][['date','cnt']].set_index('date').index
            res_week.append(self.metrics_report(ans.iloc[:90], self.longtime_models[i].predict(dates[:90])))
            res_month.append(self.metrics_report(ans.iloc[:30], self.longtime_models[i].predict(dates[:30])))
            res_quarter.append(self.metrics_report(ans.iloc[:7], self.shorttime_models[i].predict(dates[:7])))

        result =pd.concat([sum(res_week)/15,
                           sum(res_month)/15,
                           sum(res_quarter)/15],
                           axis=1)
        result.columns = ['Week_mean','Week_median','Month_mean','Month_median','Quarter_mean','Quarter_median']
        return result


    def save(self, path):
        joblib.dump((self.shorttime_models,self.longtime_models), path)
        return self

    def load(self, path):
        self.shorttime_models,self.longtime_models = joblib.load(path)
        return self

# Проверка работоспособности класса

## Тестовые данные (Просмотр)

In [ ]:
sales = pd.read_csv('data/shop_sales_test.csv')
sales.tail()

,item_id,store_id,date_id,cnt
4225,STORE_1_727,STORE_1,1909,1
4226,STORE_1_727,STORE_1,1910,1
4227,STORE_1_727,STORE_1,1911,4
4228,STORE_1_727,STORE_1,1912,1
4229,STORE_1_727,STORE_1,1913,5


In [ ]:
calendar = pd.read_csv('data/shop_sales_dates_test.csv')
calendar.tail()

,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,date_id,CASHBACK_STORE_1,CASHBACK_STORE_2,CASHBACK_STORE_3
145,2016-06-15,11620,Wednesday,5,6,2016,NaN,NaN,NaN,NaN,1965,1,0,1
146,2016-06-16,11620,Thursday,6,6,2016,NaN,NaN,NaN,NaN,1966,0,0,0
147,2016-06-17,11620,Friday,7,6,2016,NaN,NaN,NaN,NaN,1967,0,0,0
148,2016-06-18,11621,Saturday,1,6,2016,NaN,NaN,NaN,NaN,1968,0,0,0
149,2016-06-19,11621,Sunday,2,6,2016,NBAFinalsEnd,Sporting,Father's day,Cultural,1969,0,0,0


In [ ]:
prices = pd.read_csv('data/shop_sales_prices_test.csv')
prices.tail()

,store_id,item_id,wm_yr_wk,sell_price
1030,STORE_1,STORE_1_584,11621,1.00
1031,STORE_1,STORE_1_586,11621,1.68
1032,STORE_1,STORE_1_587,11621,2.48
1033,STORE_1,STORE_1_714,11621,1.58
1034,STORE_1,STORE_1_727,11621,3.98


Заодно посмотрим, как класс эти данные предобрабатывает

In [ ]:
obj = TimeSeriosProcessor(sales, prices, calendar)

In [ ]:
obj.data

,item_id,store_id_x,date_id,cnt,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,CASHBACK_STORE_1,CASHBACK_STORE_3,store_id_y,sell_price
0,STORE_2_085,STORE_2,1820,11,2016-01-22,11551,7,1,2016,NaN,NaN,NaN,NaN,0,0,STORE_2,0.98
1,STORE_2_085,STORE_2,1821,4,2016-01-23,11552,1,1,2016,NaN,NaN,NaN,NaN,0,0,STORE_2,0.98
2,STORE_2_085,STORE_2,1822,12,2016-01-24,11552,2,1,2016,NaN,NaN,NaN,NaN,0,0,STORE_2,0.98
3,STORE_2_085,STORE_2,1823,1,2016-01-25,11552,3,1,2016,NaN,NaN,NaN,NaN,0,0,STORE_2,0.98
4,STORE_2_085,STORE_2,1824,4,2016-01-26,11552,4,1,2016,NaN,NaN,NaN,NaN,0,0,STORE_2,0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4225,STORE_1_727,STORE_1,1909,1,2016-04-20,11612,5,4,2016,NaN,NaN,NaN,NaN,0,0,STORE_1,3.98
4226,STORE_1_727,STORE_1,1910,1,2016-04-21,11612,6,4,2016,NaN,NaN,NaN,NaN,0,0,STORE_1,3.98
4227,STORE_1_727,STORE_1,1911,4,2016-04-22,11612,7,4,2016,NaN,NaN,NaN,NaN,0,0,STORE_1,3.98
4228,STORE_1_727,STORE_1,1912,1,2016-04-23,11613,1,4,2016,NaN,NaN,NaN,NaN,0,0,STORE_1,3.98


## Обучение на тестовых данных

In [ ]:
obj.fit()

## Оценка обучения класса

In [ ]:
obj.eval()

,Week_mean,Week_median,Month_mean,Month_median,Quarter_mean,Quarter_median
maes,14.292954,11.850409,13.544444,11.425469,17.285029,15.666565
mses,193.550211,84.846204,172.852415,79.708147,260.304290,158.304312


## Сохранение и подгрузка весов в класс

In [ ]:
obj.save('model_list.joblib')

In [ ]:
obj.load('model_list.joblib')

## Прогнозы (Предсказания класса)

In [ ]:
obj.predict('Week')

,STORE_2_085,STORE_2_043,STORE_2_054,STORE_2_325,STORE_2_090,STORE_2_252,STORE_2_376,STORE_2_391,STORE_2_524,STORE_2_555,...,STORE_1_252,STORE_1_339,STORE_1_546,STORE_1_547,STORE_1_555,STORE_1_584,STORE_1_586,STORE_1_587,STORE_1_714,STORE_1_727
2016-04-25,5.684336,0.691498,3.116975,5.914889,90.083003,48.259232,6.784863,3.108483,0.073934,29.231609,...,17.665852,4.734045,0.552325,14.01064,26.432427,1.499999,29.570391,26.338755,16.262449,2.307731
2016-04-26,5.684336,0.691498,3.116975,5.914889,85.714491,45.211153,6.272733,2.319992,0.073934,28.214354,...,13.442980,4.734045,-0.347893,14.01064,22.719863,1.499999,25.182093,21.326470,20.730952,2.692225
2016-04-27,5.684336,0.691498,3.116975,5.914889,86.563474,44.211527,4.381447,2.373266,0.073934,27.645506,...,12.536172,4.734045,0.213403,14.01064,20.815084,1.499999,28.494046,21.773795,17.260797,2.153813
2016-04-28,5.684336,0.691498,3.116975,5.914889,95.084119,46.062044,4.587935,1.435396,0.073934,27.821352,...,13.709416,4.734045,-0.148326,14.01064,23.269112,1.499999,27.653751,13.992595,21.036989,2.076843
2016-04-29,5.684336,0.691498,3.116975,5.914889,128.160399,43.465650,5.577798,3.074035,0.073934,32.623671,...,18.127454,4.734045,-0.336634,14.01064,26.230948,1.499999,33.827551,17.201412,25.107795,3.571489
2016-04-30,5.684336,0.691498,3.116975,5.914889,158.378950,62.548295,6.783331,1.621809,0.073934,38.837320,...,26.151455,4.734045,1.470409,14.01064,37.205003,1.499999,42.584801,23.906847,27.220204,4.857123
2016-05-01,5.684336,0.691498,3.116975,5.914889,139.655817,65.849711,9.645843,2.384424,0.073934,36.795472,...,24.586319,4.734045,1.134523,14.01064,31.191296,1.499999,37.922326,31.467606,19.392284,7.285756


In [ ]:
obj.predict('Month')

,STORE_2_085,STORE_2_043,STORE_2_054,STORE_2_325,STORE_2_090,STORE_2_252,STORE_2_376,STORE_2_391,STORE_2_524,STORE_2_555,...,STORE_1_252,STORE_1_339,STORE_1_546,STORE_1_547,STORE_1_555,STORE_1_584,STORE_1_586,STORE_1_587,STORE_1_714,STORE_1_727
2016-04-25,5.434245,-0.230737,2.713067,4.041101,104.141189,47.418620,6.237352,1.467412,0.753084,31.258640,...,18.364852,3.908305,0.596778,17.204234,25.938495,1.889796,29.923378,27.407191,19.497443,3.751572
2016-04-26,6.629731,0.596115,3.238762,5.154002,72.362712,51.747314,4.790383,1.813229,0.433103,26.425917,...,17.620269,3.801360,0.268616,16.981576,22.283905,1.535147,25.888086,20.166880,21.444695,3.047066
2016-04-27,5.956404,0.541560,4.479844,7.074068,77.019695,55.617777,4.716282,2.131920,1.131089,18.638631,...,11.247731,5.174956,1.128169,15.331460,17.675966,1.843882,26.107836,15.599529,17.107495,0.387314
2016-04-28,5.744243,0.805464,2.082196,6.685993,97.309884,41.595910,3.736001,1.916458,0.375716,20.208389,...,16.441810,5.186261,0.818675,17.954448,21.090365,2.398236,33.254298,16.453652,23.299878,1.590959
2016-04-29,7.194626,1.558286,2.800697,4.545222,123.315850,40.295284,6.147749,1.295493,1.603131,25.374019,...,19.999661,3.341345,1.371691,20.621648,24.462758,2.386550,36.482944,21.041274,23.714411,4.313726
2016-04-30,6.579441,0.119642,3.578757,4.416479,147.655368,42.206811,5.431395,2.159703,0.726251,32.616877,...,27.295661,5.841610,0.834220,19.069150,35.033201,1.685029,39.678751,28.278263,24.508963,4.154288
2016-05-01,5.521091,0.734380,4.212198,6.803999,134.318190,38.447902,6.176463,1.722412,1.466705,38.782701,...,22.609195,5.658853,0.867333,14.051833,28.718038,1.200308,36.411898,29.570239,20.742707,6.064136
2016-05-02,7.090090,0.330241,2.665161,6.749665,98.736166,32.502683,5.514534,2.220251,1.266888,35.977767,...,18.675493,4.223680,0.963598,16.052095,28.063221,1.489182,26.474426,26.904650,19.721146,4.411556
2016-05-03,6.899375,0.578854,2.137890,5.263427,79.431856,42.929052,6.418983,1.421957,1.412258,30.925787,...,12.157373,4.270834,0.951328,18.332464,20.118521,2.056295,24.428444,20.354079,21.850133,2.655146
2016-05-04,5.923811,1.476611,3.907212,4.742203,75.536569,45.483593,4.886007,1.950751,1.876085,23.027803,...,8.753460,6.137143,1.601284,15.579737,17.634891,2.101470,23.400761,15.497087,19.107969,1.804069


In [ ]:
obj.predict('Quarter')

,STORE_2_085,STORE_2_043,STORE_2_054,STORE_2_325,STORE_2_090,STORE_2_252,STORE_2_376,STORE_2_391,STORE_2_524,STORE_2_555,...,STORE_1_252,STORE_1_339,STORE_1_546,STORE_1_547,STORE_1_555,STORE_1_584,STORE_1_586,STORE_1_587,STORE_1_714,STORE_1_727
2016-04-25,5.434245,-0.230737,2.713067,4.041101,104.141189,47.418620,6.237352,1.467412,0.753084,31.258640,...,18.364852,3.908305,0.596778,17.204234,25.938495,1.889796,29.923378,27.407191,19.497443,3.751572
2016-04-26,6.629731,0.596115,3.238762,5.154002,72.362712,51.747314,4.790383,1.813229,0.433103,26.425917,...,17.620269,3.801360,0.268616,16.981576,22.283905,1.535147,25.888086,20.166880,21.444695,3.047066
2016-04-27,5.956404,0.541560,4.479844,7.074068,77.019695,55.617777,4.716282,2.131920,1.131089,18.638631,...,11.247731,5.174956,1.128169,15.331460,17.675966,1.843882,26.107836,15.599529,17.107495,0.387314
2016-04-28,5.744243,0.805464,2.082196,6.685993,97.309884,41.595910,3.736001,1.916458,0.375716,20.208389,...,16.441810,5.186261,0.818675,17.954448,21.090365,2.398236,33.254298,16.453652,23.299878,1.590959
2016-04-29,7.194626,1.558286,2.800697,4.545222,123.315850,40.295284,6.147749,1.295493,1.603131,25.374019,...,19.999661,3.341345,1.371691,20.621648,24.462758,2.386550,36.482944,21.041274,23.714411,4.313726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-07-19,7.069415,0.694791,2.403887,5.871883,83.605791,44.214749,5.966684,1.522952,4.465549,23.820609,...,11.691076,5.119833,2.391098,14.005175,22.622348,1.559687,22.251018,19.640190,21.092551,3.776801
2016-07-20,7.165642,0.625238,2.560678,5.863878,77.691006,40.904365,5.903424,1.520363,4.675277,25.537264,...,9.105126,5.180999,2.349884,14.034127,21.062712,1.390568,24.048434,16.635146,21.089226,2.220355
2016-07-21,7.531916,0.677055,3.830874,5.877226,94.395599,36.299807,5.866893,1.517796,4.666569,29.395470,...,11.587245,4.127317,2.300907,14.763402,23.778504,1.399552,30.576400,18.173724,21.090368,1.683909
2016-07-22,7.211474,0.732018,3.790954,5.885026,121.113638,41.748387,5.948691,1.515252,4.508773,32.805629,...,17.275326,4.289182,2.271029,14.621337,24.255784,1.568790,35.633258,23.071335,21.092559,2.563292


# Задание 2: Отчёт по работе

## Какие методы предобработки данных вы использовали?

**Данные предобрабатывались в основном при помощи библиотеки `pandas`:**

- объединение таблтц - функции `pd.merge` и `pd.concat`

- получение обрабатымаемого формата дат - функция `pd.to_datetime`

- заполнение пропусков в оъединённой табличке - комбинация методов `ffill()` и `bfill()`

- анализ табличных данных - методы `corr`, `var`, `describe`

**Дополнительно**: библиотеки `numpay`, `matplotlib`


## Какие модели пробовали? Почему пробовали именно их?


- [x] **ARIMA**

>Это какая-то база, её очень часто берут в первую очередь (я, в общем-то, сделала также), модель хорошо улавливающая краткосрочные закономерности в данных, в моём случае это сработало именно так (модель взята в качестве предсказателя краткосрочных прогнозов). Я попробовала штук 6 вариаций модели, стремясь как-то усреднить параметры, чтобы они хорошо отражали все ряды в совокупности

- [ ] **SARIMAX**

> Попробовала её как более сложную модельку из семейства. Визуально даже думала, что она войдет в финальный класс, но одна из вариаций ARIMA всё же её обошла


- [x] **AutoETS** (экспоненциальное сглаживание)

> Выбрала её из-за автоподбора параметров, так как у меня было много разнородных рядов, которые хотелось обработать с помощью одной модельки (в общем это дало свои плоды, но странным образом. Я расчитывала, что моделька будет хороша на краткосрочных прогнозах, но в итоге я взяла её как долгосрочный предсказатель)

- [ ] **ThetaForecaster** и **ExponentialSmoothing**

> Двойное экспоненциальное сглаживание (с учетом тренда) и тройное экспоненциальное сглаживание (с учётом тренда и сезонности). Попробовала их, как модельки экспоненциального сглаживания, но они показала крайне слабые результаты (на порядок ниже других моделей), может я ошиблась где-то в настройках моделек


- [ ]  **Prophet**

>Пробовала его из-за того, что он учитывает праздники (как будто интуитивно и по визуальзациям в нашей задаче продажи могли сильно зависить от распределения праздников). Он показал хорошие результаты (особенно визуально), но всё же проиграл моделькам поменьше

- [ ] **TBATS**

> Пробовала в качестве сложной модели, но отбросила даже не доучив из-за того, что потребовалось много времени на обучения. Мне показалось, что модель всё же оказалась избыточной для этой задачи (но я честно пробовала, доучила, чтобы получить результаты, проигравшие AutoETS)

## Как вы проверяете качество модели? На каких данных? Какие метрики используюте? Чем обусловлен выбор именно этих метрик?

**В процессе исследований оценка происходила следующим образом:**

- В тест отложила 3 месяца (где-то 10% от данных из конца)

- Метрики применяла к части этой отложенной выборки из `90`/`30`/`7` наблюдений соответственно для предсказаний на `квартал`/`месяц`/`неделю`

> `Зачем откладывала больше? Не подумала`

- Из метрик пробовала всё, что завещали нам семинары: *средние* и *мединные* прогнозы `MAE`, `MSE`,`MAPE`, `R2-score` (квантили убрала всё же).

>`Не всё было обязательно в контексте задачи, но при исследованиях грех не пробовать`

**В процессе оценки модели на тестовых данных лишнее отсеялось:**

- Оценивала почти на полностью на всех данных (90 экзкмпляров из 94)

- Метрики применялись также, как и в исследовании, к первым `90`/`30`/`7` наблюдений из выборки для тех же периодов предсказаний (`квартал`/`месяц`/`неделя`)

- Из метрик отсеяла `R2-score` и `MAPE`, остались соответственно *средние* и *мединные* прогнозы `MAE`, `MSE`. Почему:


- [x] **MSE**

> Самое простое часто и есть то, что нам нужно. В данных нет особенно заметных выбросов, дисперсия тоже не запредельная, так что эта функция ошибки кажется довольно объективной

- [x] **MAE**

> Да, больших выбросов нет, но есть большая десперсия в продажах разных товаров (одного артикула в среднем продают 1-2 штуки, а другого 100-125). Как будто в усреднении прогноза MAE это немного стабилизирует

- [ ] **MAPE**

> В данных было очень много измерений, близких к нулю, функция просто не давала мне интерпритируемых результатов

- [ ] **R2-score**

> Было полезно посмотреть на влияние дисперсии между тестовыми и предсказанными значениями, но значения бвли очень близкими между разными прогнозами, поэтому решила её убрать




## Какое итоговое качество модели на тестовом датасете?

In [ ]:
sales = pd.read_csv('data/shop_sales.csv')
prices = pd.read_csv('data/shop_sales_prices.csv')
calendar = pd.read_csv('data/shop_sales_dates.csv')
train = TimeSeriosProcessor(sales, prices, calendar)
train.fit()

In [ ]:
train.save('final.joblib')

In [ ]:
obj.load('final.joblib')
obj.eval()

,Week_mean,Week_median,Month_mean,Month_median,Quarter_mean,Quarter_median
maes,20.087164,15.563251,18.405636,14.100649,16.178971,14.401450
mses,436.537382,153.332679,368.398123,125.289242,260.916707,128.904271
